# Snowflake

Existen alternativas multinube de cara a no ligarse a un proveedor de por vida. [Snowflake](https://www.snowflake.com/es/) es sin duda el líder en este ámbito en lo que a sistemas RDBMS destinados a analítica se refiere. Su versión de prueba nos permite crear un entorno válido por un mes e interactuar con este como si de una base de datos convencional se tratara.

[Crear cuenta de prueba](https://signup.snowflake.com/)

Una vez dispongáis de los datos para conectaros solo tendréis que configurar el acceso programático. Snowflake dispone de un conector propio que debería facilitarnos la tareas. Además es compatible con [Pandas](https://quickstarts.snowflake.com/guide/getting_started_with_snowpark_pandas/index.html#0).

### Variables de entorno

Podemos registrar las credenciales de conexión en un fichero _.env_ y cargar en tiempo de ejecución las variables de entorno para evitar exponer nuestras credenciales cuando versionemos el código.

Para ello es necesario que el fichero contenga los datos:
```
SNOWFLAKE_USER=<user_name>
SNOWFLAKE_PASS=<password>
SNOWFLAKE_ACCOUNT=<account>
```

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

Asi simplemente podemos recurrir a las variables de entorno en tiempo de ejecución.

In [ ]:
import os
import snowflake.connector

con = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASS"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    database="SNOWFLAKE_SAMPLE_DATA"
)

In [2]:
import pandas as pd

customers = pd.read_sql_query("SELECT * FROM TPCH_SF10.CUSTOMER", con=con)
customers.head(4)

/tmp/ipykernel_10982/957583243.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customers = pd.read_sql_query("SELECT * FROM TPCH_SF10.CUSTOMER", con=con)


,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,285001,Customer#000285001,4FeDDwg7xsqSVT,15,25-854-361-6981,3057.88,MACHINERY,"ve the regular, pending packages affix caref"
1,285002,Customer#000285002,UbQAgu44NZOvMadwuhc8IKs1fYIMz5ZNmlRYzyl,11,21-625-459-4267,-190.59,MACHINERY,ress deposits. furiously even packages sleep c...
2,285003,Customer#000285003,"Mug,CjxX0lpHalGRJeG3RCGEdCJFZXAYQ6IvlW4",12,22-998-713-3526,6636.14,AUTOMOBILE,"ideas. regular, special platelets nag furiously"
3,285004,Customer#000285004,rQ4SNwFDmtrNKgzwlxQ219j5Qef A,3,13-245-434-3165,2607.20,BUILDING,nal ideas. requests across the regular deposit...


Tenéis más información sobre el conjunto de datos [aquí](https://docs.snowflake.com/en/user-guide/sample-data-tpch). Es importante conocer las limitaciones de nuestro sistema ya que podemos tener problemas con el tamaño de algunas tablas.

```python
orders = pd.read_sql_query("SELECT * FROM TPCH_SF1000.ORDERS", con=con)
orders.head(4)
```

# Abstracciones DataFrames

Existen soluciones que nos permiten interactuar empleando las APIs conocidas (Pandas) pero impactando directamente sobre la base de datos con queries. Hay que prestar atención porque las consultas no se lanzan hasta que se requiere pintar los resultados (`.show()`, `.collect()`, `.to_pandas()`)

In [ ]:
from snowflake.snowpark import Session

connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASS"),
    "database" : "SNOWFLAKE_SAMPLE_DATA"
}  

new_session = Session.builder.configs(connection_parameters).create()  

Una vez iniciada la sesión podemos vincular variables locales a tablas en la base de datos.

In [12]:
df_region = new_session.table("TPCH_SF1.region")

Unicamente registra el nexo entre la variable y la tabla, de manera que si quisiéramos obtener esos datos dispondríamos de la consulta que los muestra.

In [13]:
df_region.queries

{'queries': ['SELECT  *  FROM (TPCH_SF1.region)'], 'post_actions': []}

`.show` lanza la consulta y nos muestra los 10 elementos iniciales de la tabla.

In [15]:
df_region.show(10)

------------------------------------------------------------------------------------
|"R_REGIONKEY"  |"R_NAME"     |"R_COMMENT"                                         |
------------------------------------------------------------------------------------
|0              |AFRICA       |lar deposits. blithely final packages cajole. r...  |
|1              |AMERICA      |hs use ironic, even requests. s                     |
|2              |ASIA         |ges. thinly even pinto beans ca                     |
|3              |EUROPE       |ly final courts cajole furiously final excuse       |
|4              |MIDDLE EAST  |uickly special accounts cajole carefully blithe...  |
------------------------------------------------------------------------------------



Mientras que `.to_pandas`retorna los datos a un objeto pandas en nuestra máquina.

In [16]:
df_region.to_pandas()

,R_REGIONKEY,R_NAME,R_COMMENT
0,0,AFRICA,lar deposits. blithely final packages cajole. ...
1,1,AMERICA,"hs use ironic, even requests. s"
2,2,ASIA,ges. thinly even pinto beans ca
3,3,EUROPE,ly final courts cajole furiously final excuse
4,4,MIDDLE EAST,uickly special accounts cajole carefully blith...


Podemos generar abstracciones mayores, por ejemplo tablas que sean joins de varias.

In [17]:
df_nation = new_session.table("TPCH_SF1.nation")

In [19]:
df_nation_region = df_nation.join(df_region, "R_REGIONKEY" == "N_REGIONKEY")
df_nation_region.queries

{'queries': ['SELECT  *  FROM (( SELECT "N_NATIONKEY" AS "N_NATIONKEY", "N_NAME" AS "N_NAME", "N_REGIONKEY" AS "N_REGIONKEY", "N_COMMENT" AS "N_COMMENT" FROM TPCH_SF1.nation) AS SNOWPARK_LEFT INNER JOIN ( SELECT "R_REGIONKEY" AS "R_REGIONKEY", "R_NAME" AS "R_NAME", "R_COMMENT" AS "R_COMMENT" FROM TPCH_SF1.region) AS SNOWPARK_RIGHT)'],
 'post_actions': []}

Una vez requerimos una acción es cuando se lanza la consulta construida.

In [20]:
df_nation_region.show(10)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"N_NATIONKEY"  |"N_NAME"   |"N_REGIONKEY"  |"N_COMMENT"                                         |"R_REGIONKEY"  |"R_NAME"     |"R_COMMENT"                                         |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0              |ALGERIA    |0              | haggle. carefully final deposits detect slyly ...  |0              |AFRICA       |lar deposits. blithely final packages cajole. r...  |
|0              |ALGERIA    |0              | haggle. carefully final deposits detect slyly ...  |1              |AMERICA      |hs use ironic, even requests. s                     |
|0              |ALGERIA    |0              | haggle. carefully final deposits detect slyl

Podemos anidar filtros y agregaciones y únicamente al incluir la acción final es cuando se lanzarán las consultas.

In [27]:
from snowflake.snowpark.functions import col

df_filtrado_algeria = df_nation_region.filter(col("N_NAME") == "ALGERIA")

In [28]:
df_filtrado_algeria

In [29]:
df_filtrado_algeria.queries

{'queries': ['SELECT  *  FROM ( SELECT  *  FROM (( SELECT "N_NATIONKEY" AS "N_NATIONKEY", "N_NAME" AS "N_NAME", "N_REGIONKEY" AS "N_REGIONKEY", "N_COMMENT" AS "N_COMMENT" FROM TPCH_SF1.nation) AS SNOWPARK_LEFT INNER JOIN ( SELECT "R_REGIONKEY" AS "R_REGIONKEY", "R_NAME" AS "R_NAME", "R_COMMENT" AS "R_COMMENT" FROM TPCH_SF1.region) AS SNOWPARK_RIGHT)) WHERE ("N_NAME" = \'ALGERIA\')'],
 'post_actions': []}

In [30]:
df_filtrado_algeria.to_pandas()

,N_NATIONKEY,N_NAME,N_REGIONKEY,N_COMMENT,R_REGIONKEY,R_NAME,R_COMMENT
0,0,ALGERIA,0,haggle. carefully final deposits detect slyly...,0,AFRICA,lar deposits. blithely final packages cajole. ...
1,0,ALGERIA,0,haggle. carefully final deposits detect slyly...,1,AMERICA,"hs use ironic, even requests. s"
2,0,ALGERIA,0,haggle. carefully final deposits detect slyly...,2,ASIA,ges. thinly even pinto beans ca
3,0,ALGERIA,0,haggle. carefully final deposits detect slyly...,3,EUROPE,ly final courts cajole furiously final excuse
4,0,ALGERIA,0,haggle. carefully final deposits detect slyly...,4,MIDDLE EAST,uickly special accounts cajole carefully blith...


In [24]:
df_nation_region.group_by("N_NAME").count()

In [25]:
df_nation_region.group_by("N_NAME").count().queries

{'queries': ['SELECT "N_NAME", count(1) AS "COUNT" FROM ( SELECT  *  FROM (( SELECT "N_NATIONKEY" AS "N_NATIONKEY", "N_NAME" AS "N_NAME", "N_REGIONKEY" AS "N_REGIONKEY", "N_COMMENT" AS "N_COMMENT" FROM TPCH_SF1.nation) AS SNOWPARK_LEFT INNER JOIN ( SELECT "R_REGIONKEY" AS "R_REGIONKEY", "R_NAME" AS "R_NAME", "R_COMMENT" AS "R_COMMENT" FROM TPCH_SF1.region) AS SNOWPARK_RIGHT)) GROUP BY "N_NAME"'],
 'post_actions': []}

In [26]:
df_nation_region.group_by("N_NAME").count().show()

------------------------
|"N_NAME"    |"COUNT"  |
------------------------
|ALGERIA     |5        |
|ARGENTINA   |5        |
|BRAZIL      |5        |
|CANADA      |5        |
|ETHIOPIA    |5        |
|GERMANY     |5        |
|INDIA       |5        |
|IRAN        |5        |
|IRAQ        |5        |
|MOZAMBIQUE  |5        |
------------------------



Al finalizar nuestras acciones cerraremos la conexión.

In [31]:
new_session.close()

Tenéis más información en la documentación oficial de Snowflake:

* [Documentación](https://docs.snowflake.com/en/user-guide-getting-started)
* [Snowpark API](https://docs.snowflake.com/en/developer-guide/snowpark/python/index)

Existe un proyecto más generalista pensado para todo tipo de bases de datos (no solo Snowflake) llamado [Apache Ibis](https://ibis-project.org/). Son abstracciones importantes cuando estamos familiarizados con el mundo de los dataframes pero queremos actuar sobre bases de datos voluminosas.